In [1]:
import pennylane as qml
import numpy as np
from matplotlib import pyplot as plt
from pennylane import qaoa as qaoa
import networkx as nx

/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/pyscf/lib/misc.py:46: H5pyDeprecationWarning: Using default_file_mode other than 'r' is deprecated. Pass the mode to h5py.File() instead.
  h5py.get_config().default_file_mode = 'a'


# Feedback-Based Quantum Optimization (FALQON)

In this demo, we'll be implement the FALQON algorithm, standing for *Feedback-based ALgorithm for Quantum Optimization*, introduced by [Magann, Rudinger, Grace & Sarovar (2021)](https://arxiv.org/pdf/2103.08619.pdf). It is similar in spirit to the [QAOA](https://arxiv.org/pdf/1411.4028.pdf), but scales with resources differently, using iterative feedback steps rather than a global optimization over parameters. We will show how to implement FALQON in PennyLane and test its performance on the **MaxClique** problem in graph theory!

### Theory

To solve combinatorial optimization problems using a quantum computer, a typical strategy is to encode the solution to the problem as the ground state of *cost Hamiltonian* $H_C$, and choose some strategy to drive the system from a known initial state into this ground state. FALQON falls under this broad scheme.

Imagine a quantum system governed by a Hamiltonian of the form $H = H_C + \beta(t) H_D$. These kinds of Hamiltonians come up quite often in the theory of [quantum control](https://quantiki.org/wiki/quantum-control-theory), which studies how we may go about driving a quantum system from one state to another. The choice of $\beta(t)$ allows us to decide which state we want a system governed by such a Hamiltonian to evolve towards. 

The time-dependent Schrodinger equation tells us that the dynamics of the system are given by:

$$
i \frac{d}{dt} |\psi(t)\rangle = (H_C + \beta(t) H_D) |\psi(t)\rangle,
$$

where we set $\hbar = 1$. Now, let us suppose that the objective of our quantum control experiment is to drive our system to the state $|\psi\rangle$: the ground state of $H_C$. Phrased differently, we would like to minimize the expectation value of $\langle H_C\rangle$ (an immediate consequence of the variational principle). Therefore, a reasonable goal is to construct our system such that the expectation decreases with time:

$$
\frac{d}{dt} \langle H_C\rangle_t = \frac{d}{dt} \langle \psi(t)|H_C|\psi(t)\rangle = i \beta(t)\langle [H_D, H_C] \rangle_t \leq 0,
$$

where we used the product rule and Schrödinger's equation. Recall that our control experiment depends on the choice of $\beta(t)$. Thus, if we pick $\beta(t) = -\langle i[H_D, H_C] \rangle_t$, so that

$$
\frac{d}{dt} \langle H_C\rangle_t = -|\langle i[H_D, H_C] \rangle_t|^2 \leq 0.
$$

then $\langle H_C \rangle$ is guaranteed to strictly decrease, as desired!

(Note that we bring the $i$ into the expectation to give a Hermitian operator.)
Using [techniques from control theory](https://arxiv.org/pdf/1304.3997.pdf), it is possible to rigorously show this will eventually drive the system into the ground state! Thus, if we evolve some initial state $|\psi_0\rangle$ under the time-evolution operator corresponding to $H$, given by $U(t) = e^{-iHt}$, then we will arrive at the ground state of $H$! We can approximately discretize $U$ using a Trotter-Suzuki decomposition, so we may implement it on a quantum circuit:

$$
U(t) \approx U_D(\beta_\ell) U_C U_D(\beta_{\ell-1}) U_C\cdots
U_D(\beta_1) U_C, \quad U_C = e^{-iH_C \Delta t}, \quad U_D(\beta_k) =
e^{-i\beta_k H_D \Delta t},
$$

where $\Delta t = t/2\ell$ and $\beta_k = \beta(2k\Delta t)$. One issue remains: for each step, we need to know the value $\beta_k$, which are themselves dependent on the state of the system, at some time. Our strategy is to use the value of $A(t) := i\langle [H_D, H_C] \rangle_t$ obtained by evaluating the circuit for the previous time-step:

$$
\beta_{k+1} = -A_k = -A(2k\Delta t).
$$

This leads immediately to the FALQON algorithm. On step $k$, we perform the following three substeps:

1. Prepare the state $|\psi_k\rangle = U_D(\beta_k) U_C \cdots U_D(\beta_1) U_C|\psi_0\rangle$,
2. Measure the expectation value $A_k = \langle i[H_C, H_D]\rangle_k$.
3. Set $\beta_{k+1} = -A_k$.

### Solving the MaxClique Problem

### QAOA Seeded By FALQON

So far, we have described the theory behind FALQON, and implemented a basic example for the MaxClique problem. As was mentioned multiple times through this tutorial, and as you probably noticed yourself, FALQON looks an awful lot like QAOA.

In the following section we will compare QAOA and QAOA seeded by FALQON.